In [ ]:
import pandas as pd
import numpy as np
import os
import pickle
import scipy.stats as st
import matplotlib.pyplot as plt
import matplotlib as mpl
from npc_sessions import DynamicRoutingSession

mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42


In [ ]:
loadpath=r'C:\Users\ethan.mcbride\OneDrive - Allen Institute\DR decoding results\by_session\blockwise_crossval_no_autos'

svc_results={}
decoder_results=os.listdir(loadpath)

In [ ]:
#TODO: add session decoder accuracy, add probe, add blockwise decoder accuracy??

balanced_accuracy_by_session={}
# loop through n units
unit_nums=np.array([0, 1])
for n_units in unit_nums:
    balanced_accuracy_by_session[n_units]={}

    balanced_accuracy_by_session[n_units]['structure']=[]
    balanced_accuracy_by_session[n_units]['probe']=[]
    balanced_accuracy_by_session[n_units]['session']=[]
    balanced_accuracy_by_session[n_units]['total_units']=[]
    balanced_accuracy_by_session[n_units]['session_balanced_accuracy']=[]
    balanced_accuracy_by_session[n_units]['session_balanced_accuracy_shuffle']=[]


#load session
for ii,ff in enumerate(decoder_results[:]):
    with open(os.path.join(loadpath,ff),'rb') as handle:
        svc_results=pickle.load(handle)
    print(ff)

    all_areas=np.unique(list(svc_results['block_ids'].keys()))
    timepoints=[0]
    label_list=['vis','aud']
    predict=['block_ids']
    n_units=svc_results['unit_numbers']
    unit_numbers=np.arange(0,len(n_units))
    n_repeats=svc_results['n_repeats']

    #extract decision functions
    
    timepoint=0
    predict=['block_ids']

    balanced_accuracy={}
    balanced_accuracy_shuffle={}

    for p in predict:
        balanced_accuracy[p]={}
        balanced_accuracy_shuffle[p]={}
        for n_units in unit_numbers:
            balanced_accuracy[p][n_units]={}
            balanced_accuracy_shuffle[p][n_units]={}
            for aa in all_areas:
                if aa in svc_results[p].keys():
                    balanced_accuracy[p][n_units][aa]={}
                    balanced_accuracy_shuffle[p][n_units][aa]={}
                    if len(svc_results[p][aa][timepoint][n_units])>0:
                        balanced_accuracy_all=[]
                        balanced_accuracy_shuffle_all=[]
                        for repeat in range(0,n_repeats):

                            balanced_accuracy[p][n_units][aa][repeat]=svc_results[p][aa][timepoint][n_units][repeat]['balanced_accuracy']
                            balanced_accuracy_shuffle[p][n_units][aa][repeat]=svc_results[p][aa][timepoint][n_units][repeat]['shuffle']['balanced_accuracy']
                            balanced_accuracy_all.append(balanced_accuracy[p][n_units][aa][repeat])
                            balanced_accuracy_shuffle_all.append(balanced_accuracy_shuffle[p][n_units][aa][repeat])

                        balanced_accuracy[p][n_units][aa]['average']=np.nanmean(np.vstack(balanced_accuracy_all))
                        balanced_accuracy_shuffle[p][n_units][aa]['average']=np.nanmean(np.vstack(balanced_accuracy_shuffle_all))

    #align results to switch times
    all_areas=np.unique(np.hstack(all_areas))
    unit_numbers=np.arange(0,len(svc_results['unit_numbers']))

    # session_id=svc_results['metadata'].session_id
    session_id=ff[:17]
    print(session_id)

    session=DynamicRoutingSession(session_id)

    p='block_ids'

    for aa in all_areas:
        for n_units in unit_numbers:
            if aa in balanced_accuracy[p][n_units].keys():
                if len(balanced_accuracy[p][n_units][aa])>0:

                    if 'probe' in aa:
                        probe=aa.split('_')[1]
                        temp_area=aa.split('_')[0]
                    else:
                        probe=None
                        temp_area=aa
                    balanced_accuracy_by_session[n_units]['structure'].append(temp_area)
                    balanced_accuracy_by_session[n_units]['probe'].append(probe)
                    balanced_accuracy_by_session[n_units]['session'].append(session_id)
                    balanced_accuracy_by_session[n_units]['total_units'].append(svc_results[p][aa]['n_units'])
                    balanced_accuracy_by_session[n_units]['session_balanced_accuracy'].append(balanced_accuracy[p][n_units][aa]['average'])
                    balanced_accuracy_by_session[n_units]['session_balanced_accuracy_shuffle'].append(balanced_accuracy_shuffle[p][n_units][aa]['average'])





In [ ]:
# ff[:17]

In [ ]:
balanced_accuracy_by_session_df=pd.DataFrame.from_dict(balanced_accuracy_by_session[0])
balanced_accuracy_by_session_df

In [ ]:
balanced_accuracy_by_session_df.query('structure=="ORBl"')['session_balanced_accuracy'].mean()

In [ ]:
balanced_accuracy_by_session_df.query('structure.str.contains("MOs")')

In [ ]:
balanced_accuracy_by_session_df.query('session_balanced_accuracy>0.7')

In [ ]:
balanced_accuracy_by_session_df['structure'].unique()

In [ ]:
block_shuffle_results=pd.read_csv(r"\\allen\programs\mindscope\workgroups\templeton\TTOC\decoding results\multi_shuffled_blocks\processed\multi_block_results_new.csv")
block_shuffle_results

In [ ]:
templ_above_chance=block_shuffle_results.query('project.str.contains("Templeton") and block_multiplier==2 and structure!="all"')['test_distribution_median'].mean()-0.5